### Just a playground for extracting geolocations from text

In [2]:
# Get the Scroll ID
import sys
import csv
from elasticsearch import Elasticsearch
es = Elasticsearch()

count = 10000
geo_file = "geonames.csv"

In [ ]:
doc = {
    'size' : count,
    "_source": ["asciiname", "coordinates"],
    "query":{
        "type" : {
            "value" : "geoname"
        }
    }
}

with open(geo_file, mode='w') as csv_file:
    fieldnames = ['id','coordinates','asciiname']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    response = es.search(index="geonames", doc_type='geoname', body=doc ,scroll='1m')
    scroll = response['_scroll_id']
    print("Scroll ID %s" % scroll)

#     orig_stdout = sys.stdout
#     f = open(geo_file, 'w')
#     sys.stdout = f
#     print("id,coordinates,asciiname") # print header

    #print first 10000
    for hit in response['hits']['hits']:
#         print("%s,\"%s\",\"%s\"" % (hit['_id'],hit['_source']['coordinates'],hit['_source']['asciiname']))
        writer.writerow({'id':hit['_id'], 'coordinates':hit['_source']['coordinates'], 'asciiname':hit['_source']['asciiname']})

    #Now use the scroll is to get the rest of the records
    while count < 11741135:  
        response = es.scroll(scroll_id = scroll, scroll = '1m')
        for hit in response['hits']['hits']:
#             print("%s,\"%s\",\"%s\"" % (hit['_id'],hit['_source']['coordinates'],hit['_source']['asciiname']))
            writer.writerow({'id':hit['_id'], 'coordinates':hit['_source']['coordinates'], 'asciiname':hit['_source']['asciiname']})

        count += 10000

#     sys.stdout = orig_stdout
#     f.close()  

    # Done outputting records!
    print("Done")

#500M Large!

In [4]:
import pandas as pd

df = pd.read_csv(geo_file)
print(df.shape)

(11741135, 3)


In [5]:
import sys; sys.executable

'/usr/local/opt/python/bin/python3.6'

In [6]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [7]:
from mordecai import Geoparser
geo = Geoparser()
geo.geoparse("I traveled from Oxford to Ottawa.")

Using TensorFlow backend.


[{'word': 'Oxford',
  'spans': [{'start': 16, 'end': 22}],
  'country_predicted': 'GBR',
  'country_conf': 0.96474487,
  'geo': {'admin1': 'England',
   'lat': '51.75222',
   'lon': '-1.25596',
   'country_code3': 'GBR',
   'geonameid': '2640729',
   'place_name': 'Oxford',
   'feature_class': 'P',
   'feature_code': 'PPLA2'}},
 {'word': 'Ottawa',
  'spans': [{'start': 26, 'end': 32}],
  'country_predicted': 'CAN',
  'country_conf': 0.83302397,
  'geo': {'admin1': 'Ontario',
   'lat': '45.41117',
   'lon': '-75.69812',
   'country_code3': 'CAN',
   'geonameid': '6094817',
   'place_name': 'Ottawa',
   'feature_class': 'P',
   'feature_code': 'PPLC'}}]

In [15]:
%%time
#Evaluate
missed = []
achieved = []

count = 0
for index, row in df.iterrows():
    #lets parse each locations!
    #print(row['asciiname'])
    res = geo.geoparse(row['asciiname'])
    if res:
        achieved.append(row['asciiname'])
    else:
        missed.append(row['asciiname'])
    
    if count % 10000 == 0:
        print("Completed %s records" % count)
    count += 1
    
print("Finally Done!")
print("%s/11741135 achieved" % len(achieved))
print("%s/11741135 missed" % len(missed))
#11,741,135 -> This will take a long time...

Completed 0 records
Completed 10000 records
Completed 20000 records
Completed 30000 records
Completed 40000 records
Completed 50000 records
Completed 60000 records
Completed 70000 records
Completed 80000 records
Completed 90000 records
Completed 100000 records
Completed 110000 records
Completed 120000 records
Completed 130000 records
Completed 140000 records
Completed 150000 records
Completed 160000 records
Completed 170000 records
Completed 180000 records
Completed 190000 records
Completed 200000 records
Completed 210000 records
Completed 220000 records
Completed 230000 records
Completed 240000 records
Completed 250000 records
Completed 260000 records
Completed 270000 records
Completed 280000 records
Completed 290000 records
Completed 300000 records
Completed 310000 records
Completed 320000 records
Completed 330000 records
Completed 340000 records
Completed 350000 records
Completed 360000 records
Completed 370000 records
Completed 380000 records
Completed 390000 records
Completed 4000

KeyboardInterrupt: 

In [16]:
print("%s/11741135 achieved" % len(achieved))
print("%s/11741135 missed" % len(missed))

130501/11741135 achieved
1065910/11741135 missed


In [27]:
%%time
#Try World Locations (Context + Uppercase)

q = ["Where is "]

wl_got = []
wl_missed = []

c = 0
with open("../Rasa/roger_nlu/location_data/world_locations_ed2.txt", 'r') as wlf:
    print("file open")
    line = wlf.readline()
    while line:
        res = geo.geoparse("%s%s" % (q,line.strip()))
        if res:
            wl_got.append(line.strip())
        else:
            wl_missed.append(line.strip())
        line = wlf.readline()
        c += 1
        
        if c % 1000 == 0:
            print("Completed %s records" % c)

# Done!
print("Finally Done!")
print("%s/%s achieved" % (len(wl_got), c))
print("%s/%s missed" % (len(wl_missed), c))

file open
Completed 1000 records
Completed 2000 records
Completed 3000 records
Completed 4000 records
Completed 5000 records
Completed 6000 records
Completed 7000 records
Completed 8000 records
Completed 9000 records
Completed 10000 records
Completed 11000 records
Completed 12000 records
Completed 13000 records
Completed 14000 records
Completed 15000 records
Completed 16000 records
Completed 17000 records
Completed 18000 records
Completed 19000 records
Completed 20000 records
Completed 21000 records
Completed 22000 records
Completed 23000 records
Completed 24000 records
Completed 25000 records
Completed 26000 records
Completed 27000 records
Completed 28000 records
Completed 29000 records
Completed 30000 records
Completed 31000 records
Completed 32000 records
Completed 33000 records
Completed 34000 records
Completed 35000 records
Completed 36000 records
Completed 37000 records
Completed 38000 records
Completed 39000 records
Completed 40000 records
Completed 41000 records
Completed 42000

In [28]:
%%time
#Try World Locations (Context + lowercase)

q = ["Where is "]

wl_got = []
wl_missed = []

c = 0
with open("../Rasa/roger_nlu/location_data/world_locations.txt", 'r') as wlf:
    print("file open")
    line = wlf.readline()
    while line:
        res = geo.geoparse("%s%s" % (q,line.strip()))
        if res:
            wl_got.append(line.strip())
        else:
            wl_missed.append(line.strip())
        line = wlf.readline()
        c += 1
        
        if c % 1000 == 0:
            print("Completed %s records" % c)

# Done!
print("Finally Done!")
print("%s/%s achieved" % (len(wl_got), c))
print("%s/%s missed" % (len(wl_missed), c))

file open
Completed 1000 records
Completed 2000 records
Completed 3000 records
Completed 4000 records
Completed 5000 records
Completed 6000 records
Completed 7000 records
Completed 8000 records
Completed 9000 records
Completed 10000 records
Completed 11000 records
Completed 12000 records
Completed 13000 records
Completed 14000 records
Completed 15000 records
Completed 16000 records
Completed 17000 records
Completed 18000 records
Completed 19000 records
Completed 20000 records
Completed 21000 records
Completed 22000 records
Completed 23000 records
Completed 24000 records
Completed 25000 records
Completed 26000 records
Completed 27000 records
Completed 28000 records
Completed 29000 records
Completed 30000 records
Completed 31000 records
Completed 32000 records
Completed 33000 records
Completed 34000 records
Completed 35000 records
Completed 36000 records
Completed 37000 records
Completed 38000 records
Completed 39000 records
Completed 40000 records
Completed 41000 records
Completed 42000

In [30]:
%%time
#Try World Locations (lowercase)

wl_got = []
wl_missed = []

c = 0
with open("../Rasa/roger_nlu/location_data/world_locations.txt", 'r') as wlf:
    print("file open")
    line = wlf.readline()
    while line:
        res = geo.geoparse("%s" % (line.strip()))
        if res:
            wl_got.append(line.strip())
        else:
            wl_missed.append(line.strip())
        line = wlf.readline()
        c += 1
        
        if c % 1000 == 0:
            print("Completed %s records" % c)

# Done!
print("Finally Done!")
print("%s/%s achieved" % (len(wl_got), c))
print("%s/%s missed" % (len(wl_missed), c))

file open
Completed 1000 records
Completed 2000 records
Completed 3000 records
Completed 4000 records
Completed 5000 records
Completed 6000 records
Completed 7000 records
Completed 8000 records
Completed 9000 records
Completed 10000 records
Completed 11000 records
Completed 12000 records
Completed 13000 records
Completed 14000 records
Completed 15000 records
Completed 16000 records
Completed 17000 records
Completed 18000 records
Completed 19000 records
Completed 20000 records
Completed 21000 records
Completed 22000 records
Completed 23000 records
Completed 24000 records
Completed 25000 records
Completed 26000 records
Completed 27000 records
Completed 28000 records
Completed 29000 records
Completed 30000 records
Completed 31000 records
Completed 32000 records
Completed 33000 records
Completed 34000 records
Completed 35000 records
Completed 36000 records
Completed 37000 records
Completed 38000 records
Completed 39000 records
Completed 40000 records
Completed 41000 records
Completed 42000

In [31]:
%%time
#Try World Locations (uppercase)

wl_got = []
wl_missed = []

c = 0
with open("../Rasa/roger_nlu/location_data/world_locations_ed2.txt", 'r') as wlf:
    print("file open")
    line = wlf.readline()
    while line:
        res = geo.geoparse("%s" % (line.strip()))
        if res:
            wl_got.append(line.strip())
        else:
            wl_missed.append(line.strip())
        line = wlf.readline()
        c += 1
        
        if c % 1000 == 0:
            print("Completed %s records" % c)

# Done!
print("Finally Done!")
print("%s/%s achieved" % (len(wl_got), c))
print("%s/%s missed" % (len(wl_missed), c))

file open
Completed 1000 records
Completed 2000 records
Completed 3000 records
Completed 4000 records
Completed 5000 records
Completed 6000 records
Completed 7000 records
Completed 8000 records
Completed 9000 records
Completed 10000 records
Completed 11000 records
Completed 12000 records
Completed 13000 records
Completed 14000 records
Completed 15000 records
Completed 16000 records
Completed 17000 records
Completed 18000 records
Completed 19000 records
Completed 20000 records
Completed 21000 records
Completed 22000 records
Completed 23000 records
Completed 24000 records
Completed 25000 records
Completed 26000 records
Completed 27000 records
Completed 28000 records
Completed 29000 records
Completed 30000 records
Completed 31000 records
Completed 32000 records
Completed 33000 records
Completed 34000 records
Completed 35000 records
Completed 36000 records
Completed 37000 records
Completed 38000 records
Completed 39000 records
Completed 40000 records
Completed 41000 records
Completed 42000

In [25]:
print("Finally Done!")
print("%s/%s achieved" % (len(wl_got), c))
print("%s/%s missed" % (len(wl_missed), c))

Finally Done!
2593/3064 achieved
471/3064 missed


In [29]:
import json
def read_locations(filename, loc_type):
    list1 = []
    with open(filename,'r') as f:
        data = json.load(f)
    temp = data[loc_type]
    for item in temp:
        location = item['name']
        list1.append(location)
    return list1
countries = read_locations('../Rasa/roger_nlu/location_data/countries.json',"countries")
states = read_locations('../Rasa/roger_nlu/location_data/states.json',"states")
cities = read_locations('../Rasa/roger_nlu/location_data/cities.json', "cities")

all_locs = countries + states + cities

In [ ]:
# 